In [ ]:
! sudo pip --no-cache-dir install face_recognition

     |████████████████████████████████| 100.2MB 1.4MB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=3e48cab14f25afc2360392e60d4714b9c9eb399cf12d7fbb6535be98d89a7f1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-dymaoxi3/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
import dlib
import imutils
import cv2
import face_recognition
import skimage.io
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
def compare_photos(img1, img2, error_message='BadInput'):
  '''
  Takes: 
    two files in format 'filename.extension' and error message
  Returns:
    True if the photos are similar
    False if the photos are different
    error_message if some of the paths is invalid
  '''
  try:
    known_image = face_recognition.load_image_file(img1)
    unknown_image = face_recognition.load_image_file(img2)
  except FileNotFoundError:
    return error_message
  try:
    biden_encoding = face_recognition.face_encodings(known_image)[0]
    unknown_encoding = face_recognition.face_encodings(unknown_image)[0]
    results = face_recognition.compare_faces([biden_encoding], unknown_encoding)
    return results[0]
  except IndexError:
    return False


In [ ]:
def add_simple_noise_opencv(img1, mean_noise=20, std_noise=3, error_message='BadInput', save_or_not=False):
  '''
  Takes:
    an image in format 'filename.jpg'
    mean value for gaussian noise
    std for gaussian noise
    error message in case sth goes wrong:(
    save_or_not Boolean variable to write noised picture ir not
  Saves an image in local repo with name 'filename_noised.png' if required
  Returns:
    noised new pic in the form of an array
  '''
  try:
    image = cv2.imread(img1)
    mu, sigma = mean_noise, std_noise
    random_noise = np.random.normal(mu, sigma, image.shape).astype(np.uint8)
    noised_photo = cv2.add(image, random_noise)
    noised_photo = cv2.cvtColor(noised_photo, cv2.COLOR_BGR2RGB)
    if save_or_not:
      new_name = 'your_photo_simply_noised.png' 
      noised_photo = cv2.cvtColor(noised_photo, cv2.COLOR_BGR2RGB)
      cv2.imwrite(new_name, noised_photo)
    return noised_photo
  except FileNotFoundError:
    return error_message

In [ ]:
def add_blur(img_path, kernel=(3, 3)):
  '''Takes image path and kernel
  Returns blurred image array
  '''
  image = cv2.imread(img_path)
  blurred_image = cv2.GaussianBlur(image, kernel, 0)
  blurred_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB)
  return blurred_image

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                    weights='imagenet')
pretrained_model.trainable = False

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
  decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
  return decode_predictions(probs, top=1)[0][0]


def add_noise_compare_save_best(image_path, indxx=208):
  '''
  Adds adversarial noise until comparison with initial photo gives False results. Saves the best pic
  '''
  pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
  pretrained_model.trainable = False
  # ImageNet labels
  decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
  image_raw = tf.io.read_file(image_path)
  image = tf.image.decode_image(image_raw)
  image = preprocess(image)
  image_probs = pretrained_model.predict(image)
  _, image_class, class_confidence = get_imagenet_label(image_probs)
  label = tf.one_hot(indxx, image_probs.shape[-1])
  label = tf.reshape(label, (1, image_probs.shape[-1]))
  perturbations = create_adversarial_pattern(image, label)
  epsilons = np.arange(0.01, 0.50, 0.001)

  for i, eps in enumerate(epsilons):
    noised_image_path = 'your_noised_photo.jpg'

    # smart noise
    adv_x = image + eps*perturbations
    adv_x = tf.clip_by_value(adv_x, -1, 1)
    arr = (adv_x[0].numpy()*0.5 + 0.5).astype(np.float64)
    mpl.image.imsave(noised_image_path, arr)
    if not compare_photos(noised_image_path, image_path):
      return round(eps, 5), perturbations[0]

    # gaussian noise
    arr = add_simple_noise_opencv(noised_image_path) 
    mpl.image.imsave(noised_image_path, arr)
    if not compare_photos(noised_image_path, image_path):
      return round(eps, 5), perturbations[0]

    # blur
    arr = add_blur(noised_image_path)
    mpl.image.imsave(noised_image_path, arr)
    if not compare_photos(noised_image_path, image_path):
      return round(eps, 5), perturbations[0]


14540800/14536120 [==============================] - 0s 0us/step


In [ ]:
def create_desired_noise(image_path, indxx=42, epsi = 0):
  pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
  pretrained_model.trainable = False
  # ImageNet labels
  decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
  image_raw = tf.io.read_file(image_path)
  image = tf.image.decode_image(image_raw)
  image = preprocess(image)
  image_probs = pretrained_model.predict(image)
  _, image_class, class_confidence = get_imagenet_label(image_probs)
  label = tf.one_hot(indxx, image_probs.shape[-1])
  label = tf.reshape(label, (1, image_probs.shape[-1]))
  perturbations = create_adversarial_pattern(image, label)
  eps = epsi

  noised_image_path = 'your_desired_noised_photo.jpg'

  # smart noise
  adv_x = image + eps*perturbations
  adv_x = tf.clip_by_value(adv_x, -1, 1)
  arr = (adv_x[0].numpy()*0.5 + 0.5).astype(np.float64)
  mpl.image.imsave(noised_image_path, arr)
  return round(eps, 5), perturbations[0]


In [ ]:
def add_noise_compare_with_second_save(image1_path, image2_path, indxx=42, to_blur=False):
  '''
  Adds adversarial noise to first until comparison with second photo gives False results. 
  Saves the best pic
  '''
  width, height, channels = cv2.imread(image1_path).shape
  dsize = (width, height)
  pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
  pretrained_model.trainable = False
  # ImageNet labels
  decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
  image_raw = tf.io.read_file(image1_path)
  image = tf.image.decode_image(image_raw)
  image = preprocess(image)
  image_probs = pretrained_model.predict(image)
  _, image_class, class_confidence = get_imagenet_label(image_probs)
  label = tf.one_hot(indxx, image_probs.shape[-1])
  label = tf.reshape(label, (1, image_probs.shape[-1]))
  perturbations = create_adversarial_pattern(image, label)
  epsilons = np.arange(0.01, 0.50, 0.001)

  # image2_raw = = tf.io.read_file(image2_path)
  # image2 = tf.image.decode_image(image2_raw)
  # image2 = preprocess(image2)

  for i, eps in enumerate(epsilons):
    noised_image_path = 'your_noised_photo.jpg'

    # smart noise
    adv_x = image + eps*perturbations
    adv_x = tf.clip_by_value(adv_x, -1, 1)
    arr = (adv_x[0].numpy()*0.5 + 0.5).astype(np.float64)
    mpl.image.imsave(noised_image_path, arr)
    if not compare_photos(noised_image_path, image2_path): 
      return round(eps, 5), perturbations[0]

    # gaussian noise
    arr = add_simple_noise_opencv(noised_image_path) 
    mpl.image.imsave(noised_image_path, arr)
    if not compare_photos(noised_image_path, image2_path):
      return round(eps, 5), perturbations[0]

    # blur
    if to_blur:
      arr = add_blur(noised_image_path)
      mpl.image.imsave(noised_image_path, arr)
      if not compare_photos(noised_image_path, image2_path):
        return round(eps, 5), perturbations[0]

In [ ]:
def noir(input_image):
  """
  Takes path to colored photo and returns 2D array of black and white image
  """
  img = cv2.imread(input_image)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_new = np.zeros_like(img)
  img_new[:,:,0] = img_gray
  img_new[:,:,1] = img_gray
  img_new[:,:,2] = img_gray
  return img_new

In [ ]:
def get_the_noise_img(input_image_path,is_save=False):
  # Get the input label of the image.
  eps, pert = create_desired_noise(input_image_path, epsi=0.5)
  noise = eps*(pert.numpy()*0.5 + 0.5)
  if is_save:
    noised_path = 'noise_img.jpg'
    mpl.image.imsave(noised_path, noise)

In [ ]:
compare_photos('/content/Screen Shot 2020-09-06 at 19.34.45.png', '/content/square-242.jpg')

True

In [ ]:
cv2.imread('/content/Screen Shot 2020-09-06 at 19.34.45.png').shape

(504, 504, 3)

In [ ]:
add_noise_compare_with_second_save('/content/Adir.jpg', '/content/Adir.jpg')

(0.125, <tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
 array([[[ 1., -1.,  1.],
         [ 1., -1.,  1.],
         [-1., -1.,  1.],
         ...,
         [ 1.,  1.,  1.],
         [-1., -1., -1.],
         [ 1.,  1.,  1.]],
 
        [[ 1., -1., -1.],
         [-1., -1., -1.],
         [-1.,  1.,  1.],
         ...,
         [ 1.,  1.,  1.],
         [-1., -1.,  1.],
         [ 1.,  1.,  1.]],
 
        [[ 1., -1.,  1.],
         [ 1., -1., -1.],
         [-1., -1.,  1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1.,  1.]],
 
        ...,
 
        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1.,  1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1.,  1., -1.]],
 
        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],
 
        [[ 1.,  1., -1.],
         [ 1.,  1.,  1.],
     

In [ ]:
compare_photos('/content/your_desired_noised_photo.jpg', '/content/Adir.jpg')

True

In [ ]:
create_desired_noise('/content/Adir.jpg', epsi=0.5)

(0.5, <tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
 array([[[ 1., -1.,  1.],
         [ 1., -1.,  1.],
         [-1., -1.,  1.],
         ...,
         [ 1.,  1.,  1.],
         [-1., -1., -1.],
         [ 1.,  1.,  1.]],
 
        [[ 1., -1., -1.],
         [-1., -1., -1.],
         [-1.,  1.,  1.],
         ...,
         [ 1.,  1.,  1.],
         [-1., -1.,  1.],
         [ 1.,  1.,  1.]],
 
        [[ 1., -1.,  1.],
         [ 1., -1., -1.],
         [-1., -1.,  1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1.,  1.]],
 
        ...,
 
        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1.,  1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1.,  1., -1.]],
 
        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],
 
        [[ 1.,  1., -1.],
         [ 1.,  1.,  1.],
       